In [1]:
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np
import requests
import os

## Screener parser

In [2]:
def take_strings(url):
    page = requests.get(url, headers = {'User-Agent': 'Mozilla/5.0'})
    soup = BeautifulSoup(page.text, 'html.parser')
    td = soup.find_all('td', class_='table-top')
    table = td[0].parent.parent
    strings = table.find_all('tr', valign='top')
    
    return strings

In [3]:
def make_table_list(strings):
    table_list = []
    strings_list = []

    for string in strings:
        cell = string.find_all('td', class_='screener-body-table-nw')
        for i in cell:
            strings_list.append(i.text)
        strings_list.pop(0)    
        table_list.append(strings_list)
        strings_list = []
        
    return table_list

In [4]:
def market_cap_handler(data):
    for i in range(len(data)):
        if 'M' in data['Market Cap, M'].loc[i]:
            data['Market Cap, M'].loc[i] = round(float(data['Market Cap, M'].loc[i][:-1]), 0)
        elif 'B' in data['Market Cap, M'].loc[i]:
            data['Market Cap, M'].loc[i] = round(float(data['Market Cap, M'].loc[i][:-1]) * 10e3, 0)
            
    return data

In [5]:
start_url = 'https://finviz.com/screener.ashx?v=111&f=cap_smallover&o=-volume&r='

pages = [1, 21, 41, 61, 81]
table_headers = ['Ticker', 'Company', 'Sector', 'Industry', 'Country', 'Market Cap, M', 'P/E', 'Price', 'Change', 'Volume']
final_table = []

for i in pages:
    url = start_url + str(i)
    strings = take_strings(url)
    final_table += make_table_list(strings)

In [6]:
data = pd.DataFrame(final_table, columns=table_headers)
data

,Ticker,Company,Sector,Industry,Country,"Market Cap, M",P/E,Price,Change,Volume
0,ZOM,Zomedica Corp.,Healthcare,Drug Manufacturers - Specialty & Generic,USA,454.93M,-,0.98,-24.92%,"469,851,780"
1,SNDL,Sundial Growers Inc.,Healthcare,Drug Manufacturers - Specialty & Generic,Canada,503.67M,-,0.66,-1.50%,"294,883,175"
2,PLUG,Plug Power Inc.,Industrials,Electrical Equipment & Parts,USA,27.05B,-,69.50,5.27%,"152,030,271"
3,GME,GameStop Corp.,Consumer Cyclical,Specialty Retail,USA,2.24B,-,31.40,57.39%,"143,953,611"
4,NAK,Northern Dynasty Minerals Ltd.,Basic Materials,Other Industrial Metals & Mining,Canada,313.26M,-,0.60,23.16%,"129,495,931"
...,...,...,...,...,...,...,...,...,...,...
95,BP,BP p.l.c.,Energy,Oil & Gas Integrated,United Kingdom,82.94B,-,24.78,-0.92%,"14,095,840"
96,ZM,"Zoom Video Communications, Inc.",Communication Services,Telecom Services,USA,102.17B,253.92,364.63,2.19%,"13,992,372"
97,XL,XL Fleet Corp.,Consumer Cyclical,Auto Parts,USA,2.86B,-,22.02,5.11%,"13,669,695"
98,LAZR,"Luminar Technologies, Inc.",Technology,Software - Application,USA,10.51B,-,33.32,5.88%,"13,533,176"


In [7]:
data = market_cap_handler(data)
data

,Ticker,Company,Sector,Industry,Country,"Market Cap, M",P/E,Price,Change,Volume
0,ZOM,Zomedica Corp.,Healthcare,Drug Manufacturers - Specialty & Generic,USA,455,-,0.98,-24.92%,"469,851,780"
1,SNDL,Sundial Growers Inc.,Healthcare,Drug Manufacturers - Specialty & Generic,Canada,504,-,0.66,-1.50%,"294,883,175"
2,PLUG,Plug Power Inc.,Industrials,Electrical Equipment & Parts,USA,270500,-,69.50,5.27%,"152,030,271"
3,GME,GameStop Corp.,Consumer Cyclical,Specialty Retail,USA,22400,-,31.40,57.39%,"143,953,611"
4,NAK,Northern Dynasty Minerals Ltd.,Basic Materials,Other Industrial Metals & Mining,Canada,313,-,0.60,23.16%,"129,495,931"
...,...,...,...,...,...,...,...,...,...,...
95,BP,BP p.l.c.,Energy,Oil & Gas Integrated,United Kingdom,829400,-,24.78,-0.92%,"14,095,840"
96,ZM,"Zoom Video Communications, Inc.",Communication Services,Telecom Services,USA,1.0217e+06,253.92,364.63,2.19%,"13,992,372"
97,XL,XL Fleet Corp.,Consumer Cyclical,Auto Parts,USA,28600,-,22.02,5.11%,"13,669,695"
98,LAZR,"Luminar Technologies, Inc.",Technology,Software - Application,USA,105100,-,33.32,5.88%,"13,533,176"


In [9]:
data['Volume'] = [x.replace(',', '') for x in data['Volume']]
data['Volume'] = data['Volume'].astype(int)
data

AttributeError: 'int' object has no attribute 'replace'

In [10]:
data['Accumulated amount'] = 0
data

,Ticker,Company,Sector,Industry,Country,"Market Cap, M",P/E,Price,Change,Volume,Accumulated amount
0,ZOM,Zomedica Corp.,Healthcare,Drug Manufacturers - Specialty & Generic,USA,455,-,0.98,-24.92%,469851780,0
1,SNDL,Sundial Growers Inc.,Healthcare,Drug Manufacturers - Specialty & Generic,Canada,504,-,0.66,-1.50%,294883175,0
2,PLUG,Plug Power Inc.,Industrials,Electrical Equipment & Parts,USA,270500,-,69.50,5.27%,152030271,0
3,GME,GameStop Corp.,Consumer Cyclical,Specialty Retail,USA,22400,-,31.40,57.39%,143953611,0
4,NAK,Northern Dynasty Minerals Ltd.,Basic Materials,Other Industrial Metals & Mining,Canada,313,-,0.60,23.16%,129495931,0
...,...,...,...,...,...,...,...,...,...,...,...
95,BP,BP p.l.c.,Energy,Oil & Gas Integrated,United Kingdom,829400,-,24.78,-0.92%,14095840,0
96,ZM,"Zoom Video Communications, Inc.",Communication Services,Telecom Services,USA,1.0217e+06,253.92,364.63,2.19%,13992372,0
97,XL,XL Fleet Corp.,Consumer Cyclical,Auto Parts,USA,28600,-,22.02,5.11%,13669695,0
98,LAZR,"Luminar Technologies, Inc.",Technology,Software - Application,USA,105100,-,33.32,5.88%,13533176,0


In [11]:
for i in range(len(data)):
    if i != 0:
        data['Accumulated amount'][i] = data['Volume'][i] + data['Accumulated amount'][i - 1]
    else:
        data['Accumulated amount'][i] = data['Volume'][i]
        
data

D:\Programs\anaconda-3\lib\site-packages\ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """
D:\Programs\anaconda-3\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


,Ticker,Company,Sector,Industry,Country,"Market Cap, M",P/E,Price,Change,Volume,Accumulated amount
0,ZOM,Zomedica Corp.,Healthcare,Drug Manufacturers - Specialty & Generic,USA,455,-,0.98,-24.92%,469851780,469851780
1,SNDL,Sundial Growers Inc.,Healthcare,Drug Manufacturers - Specialty & Generic,Canada,504,-,0.66,-1.50%,294883175,764734955
2,PLUG,Plug Power Inc.,Industrials,Electrical Equipment & Parts,USA,270500,-,69.50,5.27%,152030271,916765226
3,GME,GameStop Corp.,Consumer Cyclical,Specialty Retail,USA,22400,-,31.40,57.39%,143953611,1060718837
4,NAK,Northern Dynasty Minerals Ltd.,Basic Materials,Other Industrial Metals & Mining,Canada,313,-,0.60,23.16%,129495931,1190214768
...,...,...,...,...,...,...,...,...,...,...,...
95,BP,BP p.l.c.,Energy,Oil & Gas Integrated,United Kingdom,829400,-,24.78,-0.92%,14095840,4480890991
96,ZM,"Zoom Video Communications, Inc.",Communication Services,Telecom Services,USA,1.0217e+06,253.92,364.63,2.19%,13992372,4494883363
97,XL,XL Fleet Corp.,Consumer Cyclical,Auto Parts,USA,28600,-,22.02,5.11%,13669695,4508553058
98,LAZR,"Luminar Technologies, Inc.",Technology,Software - Application,USA,105100,-,33.32,5.88%,13533176,4522086234


## Company page parser